In [2]:
import json
import glob
import time
import re
from glob2 import glob

In [3]:
def read_Data(data_file, data_json, label_file):
    # open train.txt
    with open(data_file) as f:
        lines = f.readlines()
    
    jsonFiles = glob(data_json + '/*.json')
    
    # extract tweet ids from train_data.json names
    regex = re.compile('[0-9]{12,22}')
    json_lst = []
    for j in jsonFiles:   
        b=regex.findall(j)
        json_lst.append(b)
    
    data_list = {}
    num = 0
    
    # extract labels
    labels = []
    if label_file:
        with open(label_file) as f:
            label_lines = f.readlines()
        for label in label_lines:
            if label.strip('\n') == 'rumour':
                labels.append(1)
            else:
                labels.append(0)
    
    for i in range(len(lines)):
        tweets = lines[i].strip('\n').split(',')
        target_tweet = {}
        retweets = []

        if [tweets[0]] in json_lst:
    
            for j in range(len(tweets)):
                filePath = data_json+tweets[j]+'.json'
                if j == 0 :
                    json_data_file = open(filePath,'r').read()
                    json_data = json.loads(json_data_file)
                    
                    target_tweet['source tweet id']=json_data['id_str']
                    target_tweet['source tweet text']=json_data['text']
       
                if j!=0 and [tweets[j]] in json_lst:
                    json_data_file = open(filePath,'r').read()
                    json_data = json.loads(json_data_file)

                    # list the retweets in a chronongical order
                    retweets.append((time.strptime(json_data['created_at'],'%a %b %d %H:%M:%S %z %Y'), json_data['text']))
                    sort_time_retweets = sorted(retweets, key=lambda x: x[0])
                    sort_retweets = [t[1] for t in sort_time_retweets]
                    target_tweet['retweets']=sort_retweets

            data_list[num] = target_tweet
            num += 1
        elif label_file:
            labels[i] = 2
            
    labels = list(filter((2).__ne__, labels))
        
    return data_list, labels


In [4]:
covid_data_file = 'covid.data.txt'
covid_data_json = 'covid-data/'

covid_data_d, covid_labels=read_Data(covid_data_file, covid_data_json,"")
print(len(covid_data_d))

with open('total_covid_data.json', 'w') as f:
    json.dump(covid_data_d, f)

15972
